(Just draft version, feel free to change)
Group Members: F. castanheira, C. Meo, M. Sala

This is our blog plus code for the Reproducibility Project of the course CS4240 Deep Learning. The paper that we selected is: "Batch Normalisation: Accelerating Deep Network Training by Reducing Internal Covariate Schift" [1].

This paper had a huge impact for the Deep Learning community, with over than 17000 citations. Training of Deep Neural Networks is often very hard, due to the vanishing/exploding gradient problem. Batch normalisation aims to correct it. The improvement in the results using batch normalisation is undoubatable. However, the paper received also some critics. For example, the term "Covariate Shift" introduce by the paper is definitively an illecit use of vocabulary and we found it quite confusing.


The focus was given to reproduce figure 1 of the paper [1].
For completness the picture together with its description is here reported.

![alt text](https://drive.google.com/uc?id=11AP4tC-HW2diQA5dC1lDIO8CVh98CE35)
     

Figure (a) represents the test accuracy on the MNIST dataset of a simple feedforward network both with and without bacth normalisation. On the x-axis there are the training steps (50K in total), and on the y-axis the accuracy. As it is possible to see, batch normalisation allow for a greater accuracy with lower training time. This is due to the distributions that the activations see. Indeed figure (b) and (c) depicts the input distribution to a typical sigmoid in the last hidden layer of the network. The 3 lines correspond to the 15, 50 and 85 [percentile](https://en.wikipedia.org/wiki/Percentile) of the inputs (for a definition of percentile see the wikipedia link). The consequence of batch normalisation is a smoother distribution which allow the network to perform better.
We decided to tackle this problem from two prospective: From one side a Pytorch version of the two networks of figure 1(a) was made, and on the other an implementation from Skratch using just Pytorch tensors for computational efficiency. Lastly, we tried to assess the robusteness of the results using a CNN on MNIST.


[1] S. Ioffe, C. Szegedy (2015). "Batch Normalisation: Accelerating Deep Network Training by Reducing Internal Covariate Schift"

Loading all the packages that are needed.

In [0]:
import os
from google.colab import drive

drive.mount('/content/gdrive')
# Here specify the path to your directory
!ls "/content/gdrive/My Drive/DeepLearning" 
root_path = 'gdrive/My Drive/DeepLearning' 
path ='/content/gdrive/My Drive/DeepLearning'
os.chdir(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
BatchNormalisation.ipynb  figure1.PNG		 percentile_ffn_NO_BN.png
clip_figure1.PNG	  FromScratch.ipynb
data			  percentile_ffn_BN.png


In [0]:
# test of drive
print(os.getcwd()) 

/content/gdrive/My Drive/DeepLearning


In [0]:
import torch 
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

import time # to save the model with the date

torch.manual_seed(1)

Loading the train and test loaders with bacthes of 60 as in the paper.

In [0]:
transform = transforms.Compose( [transforms.ToTensor()] )                          
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=60,     
                                          shuffle=True, num_workers=1)
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                     download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=60,
                                         shuffle=False, num_workers=1)

classes = ('0', '1', '2', '3',
           '4', '5', '6', '7', '8', '9')

The following two classes are inspired by section ... of the paper. 

In [0]:
# This is the net with Batch Normalisation  
class Net_BN(nn.Module):
    def __init__(self):
        super(Net_BN, self).__init__(lr=0.2, momentum=0.9)
        self.fc1 = nn.Linear(28 * 28, 100)             # first hidden layer
        self.bn1 = nn.BatchNorm1d(100) 
        self.fc2 = nn.Linear(100, 100)                 # second hidden layer
        self.bn2 = nn.BatchNorm1d(100)
        self.fc3 = nn.Linear(100, 100)                 # third hidden layer
        self.bn3 = nn.BatchNorm1d(100)                
        self.fc4 = nn.Linear(100, 10)                  # output layer        
        self.lr = lr
        self.momentum = momentum

        self.init_weights = self.fc1.weight.data      # to print a typical initialisation

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(self.parameters(), lr=model.lr, momentum=model.momentum)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)
        self.name = 'ffn_BN'

    def forward(self, x):
        x = x.to(self.device)
        x = torch.flatten(x, start_dim=1)
        x = torch.sigmoid(self.bn1(self.fc1(x)))
        x = torch.sigmoid(self.bn2(self.fc2(x)))
        x = self.bn3(self.fc3(x))     # dimension is (60, 100)

        self.activation = x[:, 0]    # input to 10th sigmoid, tensor of size 60
        self.activations = x
        x = torch.sigmoid(x)
        
        x = self.fc4(x)
        return x

# This is the net without Batch Normalisation 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__(lr=0.2, momentum=0.9)
        self.fc1 = nn.Linear(28 * 28, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 100)
        self.fc4 = nn.Linear(100, 10)
        
        self.lr = lr
        self.momentum = momentum

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(self.parameters(), lr=model.lr, momentum=model.momentum)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)
        self.name = 'ffn_NO_BN'

    def forward(self, x):
        x = x.to(self.device)
        x = torch.flatten(x, start_dim=1)
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))

        x = self.fc3(x)

        self.activation = x[:, 0] 
        self.activations = x
        x = torch.sigmoid(x)

        x = self.fc4(x)
        return x  

net_BN = Net_BN()
net    = Net()

NameError: ignored

This has the porpuse of proving that the weights are already initialized to small values by default. Therefore we did not implement a different initialisation procedure.

In [0]:
init_weights = net_BN.init_weights
print(init_weights)
print(torch.max(init_weights))
print(torch.min(init_weights))

# if we wan to initialize the weight differently

## takes in a module and applies the specified weight initialization
def weights_init(m):
    '''Takes in a module and initializes all linear layers with weight
        values taken from a normal distribution.'''

    classname = m.__class__.__name__
    # for every Linear layer in a model
    if classname.find('Linear') != -1:
        y = m.in_features
    # m.weight.data shoud be taken from a normal distribution
        m.weight.data.normal_(0.0,1/np.sqrt(y))
    # m.bias.data should be 0
        m.bias.data.fill_(0)


tensor([[ 0.0184, -0.0158, -0.0069,  ...,  0.0068, -0.0041,  0.0025],
        [-0.0274, -0.0224, -0.0309,  ..., -0.0029,  0.0013, -0.0167],
        [ 0.0282, -0.0095, -0.0340,  ..., -0.0141,  0.0056, -0.0335],
        ...,
        [-0.0170, -0.0294, -0.0351,  ..., -0.0320, -0.0291, -0.0083],
        [ 0.0207, -0.0126,  0.0167,  ..., -0.0350, -0.0347, -0.0292],
        [ 0.0182,  0.0104,  0.0114,  ..., -0.0278, -0.0205,  0.0123]])
tensor(0.0357)
tensor(-0.0357)


The function *train* is used to avoid repetition of code. It works for both the network with and without bacth normalisation. By default loss is printed every 1000 minibatches.
It is very important to use the command *model.eval()* when the purpose is to test the model (even if toch.no_grad() is used). The reason is that it tells to *batch_norm* to operate in testing mode.

In [0]:
def train(model, tainloader, testloader, epochs = 50, LOSS_EVERY=1000, TEST_EVERY=1000):
  '''This function train the model on the data provided by trainloader.
   Test on testloader every TEST_EVERY. 
  '''
  print('...Training starts...')

  #model.apply(weights_init)
  model.train()
  print(model)
  accuracies = []
  activation_list = []
  all_activations = []
  training_steps = 0
  for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        training_steps += 1
        model.optimizer.zero_grad()
        outputs = model(inputs)
        loss = model.criterion(outputs, labels.long().to(model.device))
        loss.backward()
        model.optimizer.step()

        running_loss += loss.item()
        if i % LOSS_EVERY == LOSS_EVERY-1:    
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

        #model.eval() 

        correct = 0
        if i % TEST_EVERY == TEST_EVERY-1:      
          with torch.no_grad():
              for (images, labels) in testloader:
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                correct += (predicted ==  labels.long().to(model.device)).sum().item()
              
              accuracies.append(100 * correct / len(testloader.dataset))   

    activation_list.append(model.activation) # appending the activation here   
    all_activations.append(model.activations) 

  timestr = time.strftime("%Y%m%d-%H%M%S")
  model.name = f'net_accuracy_{accuracies[-1]}_lr={model.lr}_{timestr}' 

  torch.save(torch.save(the_model.state_dict() ) #, PATH)
  print('......Training is done......')
  return accuracies, activation_list, all_activations 

Training the feedforward network without batch normalisation

In [0]:
accuracies, activation_list, all_activations = train(net, trainloader, testloader, epochs = 50)

........Training starts........
Net(
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=10, bias=True)
  (criterion): CrossEntropyLoss()
)
[1,  1000] loss: 231.235
[2,  1000] loss: 225.665
[3,  1000] loss: 135.182
[4,  1000] loss: 70.372
[5,  1000] loss: 47.197
[6,  1000] loss: 36.898
[7,  1000] loss: 29.893
[8,  1000] loss: 25.234
[9,  1000] loss: 21.866
[10,  1000] loss: 19.173
[11,  1000] loss: 16.918
[12,  1000] loss: 15.330
[13,  1000] loss: 13.875
[14,  1000] loss: 12.659
[15,  1000] loss: 11.558
[16,  1000] loss: 10.714
[17,  1000] loss: 9.913
[18,  1000] loss: 9.362
[19,  1000] loss: 8.577
[20,  1000] loss: 7.923
[21,  1000] loss: 7.527
[22,  1000] loss: 7.022
[23,  1000] loss: 6.572
[24,  1000] loss: 6.339
[25,  1000] loss: 5.816
[26,  1000] loss: 5.408
[27,  1000] loss: 5.160
[28,  1000] loss: 4.78

training the feedforward network with batch normalisation

In [0]:
accuracies_BN, activations_BN, all_activations_BN = train(net_BN, trainloader, testloader, epochs = 50)

........Training starts........
Net_BN(
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=100, out_features=100, bias=True)
  (bn3): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=100, out_features=10, bias=True)
  (criterion): CrossEntropyLoss()
)
[1,  1000] loss: 46.216
[2,  1000] loss: 24.424
[3,  1000] loss: 18.589
[4,  1000] loss: 15.168
[5,  1000] loss: 12.691
[6,  1000] loss: 10.862
[7,  1000] loss: 9.546
[8,  1000] loss: 8.209
[9,  1000] loss: 7.178
[10,  1000] loss: 6.504
[11,  1000] loss: 5.912
[12,  1000] loss: 5.227
[13,  1000] loss: 4.839
[14,  1000] loss: 4.449
[15,  1000] loss: 4.178
[16,  1000] loss: 3.644
[17,  1000] loss: 3.392
[1

**plot_accuracy** and **plot_percentile** are functions that serve to reproduce the plots in figure 1 of [1].

In [0]:
def plot_accuracy(accuracies, accuracies_BN, title='', save=False, fname=''):
  plt.figure()
  plt.title(title)
  plt.plot(np.array(accuracies)/100, '--k', label='Without BN')
  plt.plot(np.array(accuracies_BN)/100, 'b',   label='With BN')
  plt.ylim([0.7, 1])
  plt.xlim([0, 50])
  plt.xticks([10, 20, 30, 40, 50], ('10K', '20K', '30K', '40K', '50K'))
  plt.legend()
  plt.grid()

  if save:
    plt.savefig(fname, dpi=200)

  plt.show()


def plot_percentile(activations, title='', debugging=False, save=False, fname=None):
    percentile_15 = []
    percentile_50 = []
    percentile_85 = []

    for tensor in activations:
      percentile_15.append(np.percentile(tensor.cpu().detach().numpy(), 15))
      percentile_50.append(np.percentile(tensor.cpu().detach().numpy(), 50))
      percentile_85.append(np.percentile(tensor.cpu().detach().numpy(), 85))      

    plt.figure()
    plt.title(title)
    plt.plot(percentile_15, label='15 percentile')
    plt.plot(percentile_50, label='50 percentile')
    plt.plot(percentile_85, label='85 percentile')
    plt.legend()
    plt.grid()

    if save:
      plt.savefig(fname, dpi=200)

    plt.show()

    if debugging:
      return percentile_15, percentile_50, percentile_85



def plot_mean_percentile(activations, title='', debugging=False, save=False, fname=None):
    percentile_15 = []
    percentile_50 = []
    percentile_85 = []

    for tensors in activations:
      percentile_15.append(np.percentile(tensors.cpu().detach().numpy(), 15))
      percentile_50.append(np.percentile(tensors.cpu().detach().numpy(), 50))
      percentile_85.append(np.percentile(tensors.cpu().detach().numpy(), 85))      

    plt.figure()
    plt.title(title)
    plt.plot(percentile_15, label=r'$15^{th}$ percentile')
    plt.plot(percentile_50, label=r'$50^{th}$ percentile')
    plt.plot(percentile_85, label=r'$85^{th}$ percentile')
    plt.legend()
    plt.grid()

    if save:
      plt.savefig(fname, dpi=200)

    plt.show()

    if debugging:
      return percentile_15, percentile_50, percentile_85

This is taking a random neuron in the last hidden layer:

In [0]:
plot_percentile(activation_list, title='NO BN')
plot_percentile(activations_BN,  title='WITH BN')

This is using np.percentile on all the last hidden layer:

In [0]:
plot_mean_percentile(all_activations, title='NO BN', save=True, fname='percentile_ffn_NO_BN')
plot_mean_percentile(all_activations_BN, title='WITH BN', save=True, fname='percentile_ffn_BN')

In [0]:
plot_accuracy(accuracies, accuracies_BN, 'FFN', save=True, fname='fnn_accuracies')

In [0]:
break
# It is suggested to implement CNN as well:
class Conv(nn.Module):
    def __init__(self):
        super(Conv, self).__init__(lr=0.02, momentum=0.99)
        self.conv1 = nn.Conv2d(1, 6, 3, padding=1) 
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 7 * 7, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(self.parameters(), lr=lr, momentum=momentum)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, x):
        x = torch.Tensor(x).to(self.device)

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 7 * 7)
        x = F.relu(self.fc1(x))
  
        x = self.fc2(x)  
        self.activation = x  
        x = F.relu(x)
        
        x = self.fc3(x)
        return x

class Conv_BN(nn.Module):
    def __init__(self):
        super(Conv_BN, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3, padding=1)
        self.conv1_bn = nn.BatchNorm2d(6) 
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3, padding=1)
        self.conv2_bn = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 7 * 7, 120)
        self.fc1_bn = nn.BatchNorm1d(120)
        self.fc2 = nn.Linear(120, 84)
        self.fc2_bn = nn.BatchNorm1d(84)
        self.fc3 = nn.Linear(84, 10)

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(self.parameters(), lr=0.01, momentum=0.9)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, x):
        x = torch.Tensor(x).to(self.device)
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 7 * 7)
        x = F.relu(self.fc1_bn(self.fc1(x)))

        x = self.fc2_bn(self.fc2(x))
        self.activations = x   
        x = F.relu(x)

        x = self.fc3(x)
        return x

conv_BN = Conv_BN()
conv = Conv()

In [0]:
accuracies_cnn, activations_cnn = train(conv, trainloader, testloader, epochs = 30)

In [0]:
accuracies_cnn_BN, activations_cnn_BN = train(conv_BN, trainloader, testloader, epochs = 30)

In [0]:
plot_accuracy(accuracies_cnn, accuracies_cnn_BN, title='CNN') # it seems convolutions outmatch batch normalisation for this "simple" CNN. Might be wrong tho.
plot_percentile(activations, 'Without BN')
plot_percentile(activations_BN, 'With BN')

In [0]:
# If we want to save the accuracies to txt

np.savetxt('accuracies.txt', accuracies)
np.savetxt('accuracies_BN.txt', accuracies_BN)
np.savetxt('accuracies_conv.txt', accuracies_cnn)
np.savetxt('accuracies_conv_BN.txt', accuracies_cnn_BN)
